# Projeto Grupo BT-G3


## **Integrantes do grupo**
- Daniel Barzilai
- Larissa Carvalho
- Maria Luisa Maia
- Pedro Rezende
- Rafael Moritz
- Vitor Oliveira

<center><img src="https://www.inteli.edu.br/wp-content/uploads/2021/08/20172028/marca_1-2.png" width="50%" height="50%"/></center>

<h1 align='center'><b>IA para Marketing: Monitoramento de campanhas utilizando processamento de linguagem natural (PLN)<b></h1>

<center><img src="https://upload.wikimedia.org/wikipedia/commons/c/c2/Btg-logo-blue.svg" width="50%" height="50%"/></center>

<h2 align='center'>O Banco BTG Pactual enfrenta um desafio na área de Marketing em entender as necessidades e demandas dos clientes de maneira fácil e rápida nas redes sociais. A solução proposta para esse problema foi o desenvolvimento de uma Inteligência Artificial utilizando processamento de linguagem natural (PLN), capaz de monitorar as campanhas de marketing, voltadas para o Instagram. O objetivo principal dessa solução é rastrear os dados em tempo real, analisar e interpretar as mensagens e comentários enviados pelos clientes na rede social, a fim de identificar as necessidades e demandas de forma precisa e eficiente.</h2>

---

# Sobre os dados

Esse projeto está utilizando dados coletados e tratados pela equipe de Automation do BTG Pactual, o qual disponibilizou o dataset. Com base nas informações dispostas nesse dataset, realizaremos insights a cerca dos comentários feitos nos posts do Instagram do próprio banco. Vale lembrar que os dados estão anonimizados e resguardados para manter a privacidade e ética com os usuários e com o banco.

# 1. Instalação / Setup

Para o início do projeto, fizemos o desenvolvimento no Google Colab, por isso temos uma célula de conexão com o Google Drive, para poder acessar os dados. Caso seja rodado no Jupyter Notebook, precisará do dataset baixado.

In [ ]:
#Conectar com o Google Drive

from google.colab import drive
drive.mount('/content/drive')

#Conectando o ambiente ao Google Drive

Mounted at /content/drive


Aqui nós fazemos as importações para tratamento dos dados, pré-processamento dos dados e modelamento do Bag of Words.

## pips

In [ ]:
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 15.3 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.2
    Uninstalling spacy-3.5.2:
      Successfully uninstalled spacy-3.5.2


In [ ]:
!pip install wordcloud

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import re

from sklearn.feature_extraction.text import CountVectorizer
import ast
from keras.preprocessing.text import Tokenizer
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalMaxPooling1D, Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
from keras.utils import pad_sequences
import tensorflow as tf
from tensorflow.keras import backend as K
import seaborn as sns

import nltk
import spacy
import gensim
import pickle
from scipy.spatial.distance import cosine
from gensim.models import KeyedVectors

O desenvolvimento da rede neural seguiu o tutorial abaixo:
https://medium.com/@jvsavietto6/classificando-texto-com-redes-neurais-artificiais-150ef448b13d

# 2. Entendimento e Tratamento dos Dados

Rodando o dataset, para analisar seu conteúdo:

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Módulo 6/Sprint 4 - Projeto/nova_base_tratada2.csv')
df

,autor,sentimento,texto_tratado
0,winthegame_of,1,"['alvarez', 'marsal', 'estar', 'conosco', 'spo..."
1,marta_bego,1,"['btgpactual', 'with', 'make_repost', 'entenda..."
2,lmviapiana,2,"['minuto', 'touro', 'ouro']"
3,ricktolledo,2,"['querer', 'saber', 'banking', 'administrar']"
4,donadabolsaoficial,0,"['início', 'guerra', 'rússia', 'ucrânia', 'vel..."
...,...,...,...
8035,perspectiveinvestimentos,2,"['excelente', 'explicação']"
8036,eduardocolares,2,"['atendar', 'telefone', 'amor', 'deus']"
8037,danielucm,2,"['grande', 'fiis', 'mercado', 'não', 'selecion..."
8038,amgcapitalinvest,1,"['erro', 'financeiro', 'eliminar', 'parar', 'o..."


In [ ]:
df.columns

Index(['autor', 'sentimento', 'texto_tratado'], dtype='object')

In [ ]:
df['texto_tratado']

0       ['alvarez', 'marsal', 'estar', 'conosco', 'spo...
1       ['btgpactual', 'with', 'make_repost', 'entenda...
2                             ['minuto', 'touro', 'ouro']
3           ['querer', 'saber', 'banking', 'administrar']
4       ['início', 'guerra', 'rússia', 'ucrânia', 'vel...
                              ...                        
8035                          ['excelente', 'explicação']
8036              ['atendar', 'telefone', 'amor', 'deus']
8037    ['grande', 'fiis', 'mercado', 'não', 'selecion...
8038    ['erro', 'financeiro', 'eliminar', 'parar', 'o...
8039    ['morning', 'call', 'não', 'aparecer', 'spotif...
Name: texto_tratado, Length: 8040, dtype: object

In [ ]:
# Supondo que seu DataFrame seja chamado de df e a coluna seja 'texto_tratado'
df['texto_tratado'] = df['texto_tratado'].str.replace("'", "")
df['texto_tratado']

0       [alvarez, marsal, estar, conosco, sportainmet,...
1       [btgpactual, with, make_repost, entendar, impa...
2                                   [minuto, touro, ouro]
3                   [querer, saber, banking, administrar]
4       [início, guerra, rússia, ucrânia, velho, apare...
                              ...                        
8035                              [excelente, explicação]
8036                      [atendar, telefone, amor, deus]
8037    [grande, fiis, mercado, não, selecionar, princ...
8038    [erro, financeiro, eliminar, parar, ostentar, ...
8039    [morning, call, não, aparecer, spotify, atuali...
Name: texto_tratado, Length: 8040, dtype: object

# 10. Word2Vec com CBOW

## Estruturação

In [ ]:
df

,autor,sentimento,texto_tratado
0,winthegame_of,1,"[alvarez, marsal, estar, conosco, sportainmet,..."
1,marta_bego,1,"[btgpactual, with, make_repost, entendar, impa..."
2,lmviapiana,2,"[minuto, touro, ouro]"
3,ricktolledo,2,"[querer, saber, banking, administrar]"
4,donadabolsaoficial,0,"[início, guerra, rússia, ucrânia, velho, apare..."
...,...,...,...
8035,perspectiveinvestimentos,2,"[excelente, explicação]"
8036,eduardocolares,2,"[atendar, telefone, amor, deus]"
8037,danielucm,2,"[grande, fiis, mercado, não, selecionar, princ..."
8038,amgcapitalinvest,1,"[erro, financeiro, eliminar, parar, ostentar, ..."


In [ ]:
cbow = '/content/drive/MyDrive/Módulo 6/Semana 5/cbow_s50/cbow_s50.txt'

In [ ]:
model_cbow = KeyedVectors.load_word2vec_format(cbow)

## Teste isolado

In [ ]:
# Testando o word2vec
wordvec_test = model_cbow['projeto']

wordvec_test

array([-0.074174, -0.152088,  0.086627, -0.224567,  0.362562,  0.130683,
       -0.089179, -0.086973,  0.309501,  0.004112, -0.308202,  0.351789,
       -0.477863,  0.050276,  0.213283,  0.159895, -0.285545, -0.08832 ,
       -0.015449,  0.014816, -0.613861,  0.502556,  0.021688,  0.369492,
        0.280691,  0.016868,  0.105584, -0.180754, -0.078456,  0.148032,
        0.36293 , -0.011634,  0.412191, -0.009049,  0.010404,  0.131242,
       -0.032483, -0.133067, -0.063802,  0.434015, -0.214768, -0.072132,
        0.045601, -0.368866,  0.502808,  0.048293, -0.254894,  0.142581,
       -0.075066,  0.015646], dtype=float32)

## Definição de função

In [ ]:
def create_sentence_vector(model, df):
    sentence_table = []
    for sentence in df['texto_tratado']:
        word_vectors = [model[word] for word in sentence if word in model]
        if len(word_vectors) > 0:
            sentence_vector = sum(word_vectors) / len(word_vectors)
        else:
            sentence_vector = [None] * 100  # Cria uma lista de 100 elementos None
        sentence_table.append((sentence, *sentence_vector[:50]))  # Adiciona apenas os primeiros 50 elementos do vetor

    column_labels = ['Frase']
    for i in range(50):
        column_labels.append(f'Vetor{i+1}')
    df_vec = pd.DataFrame(sentence_table, columns=column_labels)

    df["sentimentoNumerico"] = df["sentimento"].replace({'NEGATIVE': -1, 'POSITIVE': 1, 'NEUTRAL': 0})

    # Definir o índice do DataFrame df_vec como o mesmo índice de df_processada['sentimentoNumerico']
    df_vec.set_index(df["sentimentoNumerico"].index, inplace=True)

    df_vec['sentimento'] = df["sentimentoNumerico"]
    df_vec = df_vec.dropna()

    return df_vec

## Teste de funções

In [ ]:
df_vec = create_sentence_vector(model_cbow, df)
df_vec

,Frase,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,Vetor6,Vetor7,Vetor8,Vetor9,...,Vetor42,Vetor43,Vetor44,Vetor45,Vetor46,Vetor47,Vetor48,Vetor49,Vetor50,sentimento
0,"[alvarez, marsal, estar, conosco, sportainmet,...",0.214823,-0.123036,0.220707,-0.079739,-0.019442,0.202300,0.073742,0.042477,0.057268,...,0.012086,-0.096122,0.148153,0.092629,-0.027504,0.149860,-0.029617,0.032065,0.190372,1
1,"[btgpactual, with, make_repost, entendar, impa...",0.220860,-0.123713,0.205813,-0.063700,-0.010433,0.195687,0.072226,0.026175,0.058045,...,0.006447,-0.076174,0.157446,0.085306,-0.001963,0.156306,-0.019182,0.035594,0.192635,1
2,"[minuto, touro, ouro]",0.265227,-0.068285,0.152235,-0.044329,-0.102729,0.141353,0.092800,0.113174,0.015783,...,0.078032,-0.202677,0.155750,0.062291,0.007038,0.134573,0.014635,0.034189,0.345674,2
3,"[querer, saber, banking, administrar]",0.189241,-0.169407,0.281181,-0.093912,-0.001968,0.181633,0.080923,0.055674,0.103454,...,0.092738,-0.160601,0.179499,0.091803,-0.063762,0.171190,-0.043618,0.030563,0.196250,2
4,"[início, guerra, rússia, ucrânia, velho, apare...",0.219838,-0.138311,0.229412,-0.057420,-0.014510,0.212875,0.058865,0.068096,0.060398,...,0.040836,-0.112662,0.170401,0.094290,-0.046000,0.168181,-0.018029,0.027667,0.180134,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8035,"[excelente, explicação]",0.190917,-0.133475,0.241675,-0.053180,0.067256,0.201138,0.034109,-0.078718,-0.066131,...,-0.082151,0.016113,0.154861,0.068700,-0.004302,0.079717,-0.028388,-0.017448,0.188785,2
8036,"[atendar, telefone, amor, deus]",0.188641,-0.119377,0.199339,-0.105448,0.023176,0.178837,0.069476,-0.004494,0.034710,...,0.034035,-0.126673,0.165176,0.080313,-0.024160,0.118848,-0.003502,0.087053,0.215656,2
8037,"[grande, fiis, mercado, não, selecionar, princ...",0.219437,-0.141124,0.217854,-0.065569,-0.020409,0.205403,0.061334,0.041665,0.063050,...,0.031438,-0.099000,0.176588,0.092546,-0.046104,0.145734,-0.019857,0.029422,0.183504,2
8038,"[erro, financeiro, eliminar, parar, ostentar, ...",0.218153,-0.146420,0.245234,-0.056113,-0.036187,0.224789,0.074320,0.045111,0.040841,...,0.035260,-0.126793,0.171115,0.095479,-0.033473,0.158747,-0.006721,0.029326,0.198914,1


In [ ]:
word2vec = df_vec.iloc[:, 1:51]
word2vec

,Vetor1,Vetor2,Vetor3,Vetor4,Vetor5,Vetor6,Vetor7,Vetor8,Vetor9,Vetor10,...,Vetor41,Vetor42,Vetor43,Vetor44,Vetor45,Vetor46,Vetor47,Vetor48,Vetor49,Vetor50
0,0.214823,-0.123036,0.220707,-0.079739,-0.019442,0.202300,0.073742,0.042477,0.057268,0.168187,...,-0.281536,0.012086,-0.096122,0.148153,0.092629,-0.027504,0.149860,-0.029617,0.032065,0.190372
1,0.220860,-0.123713,0.205813,-0.063700,-0.010433,0.195687,0.072226,0.026175,0.058045,0.163238,...,-0.305640,0.006447,-0.076174,0.157446,0.085306,-0.001963,0.156306,-0.019182,0.035594,0.192635
2,0.265227,-0.068285,0.152235,-0.044329,-0.102729,0.141353,0.092800,0.113174,0.015783,0.202198,...,-0.193025,0.078032,-0.202677,0.155750,0.062291,0.007038,0.134573,0.014635,0.034189,0.345674
3,0.189241,-0.169407,0.281181,-0.093912,-0.001968,0.181633,0.080923,0.055674,0.103454,0.144342,...,-0.254610,0.092738,-0.160601,0.179499,0.091803,-0.063762,0.171190,-0.043618,0.030563,0.196250
4,0.219838,-0.138311,0.229412,-0.057420,-0.014510,0.212875,0.058865,0.068096,0.060398,0.156307,...,-0.269136,0.040836,-0.112662,0.170401,0.094290,-0.046000,0.168181,-0.018029,0.027667,0.180134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8035,0.190917,-0.133475,0.241675,-0.053180,0.067256,0.201138,0.034109,-0.078718,-0.066131,0.187608,...,-0.321192,-0.082151,0.016113,0.154861,0.068700,-0.004302,0.079717,-0.028388,-0.017448,0.188785
8036,0.188641,-0.119377,0.199339,-0.105448,0.023176,0.178837,0.069476,-0.004494,0.034710,0.150081,...,-0.194663,0.034035,-0.126673,0.165176,0.080313,-0.024160,0.118848,-0.003502,0.087053,0.215656
8037,0.219437,-0.141124,0.217854,-0.065569,-0.020409,0.205403,0.061334,0.041665,0.063050,0.167985,...,-0.300234,0.031438,-0.099000,0.176588,0.092546,-0.046104,0.145734,-0.019857,0.029422,0.183504
8038,0.218153,-0.146420,0.245234,-0.056113,-0.036187,0.224789,0.074320,0.045111,0.040841,0.150971,...,-0.270532,0.035260,-0.126793,0.171115,0.095479,-0.033473,0.158747,-0.006721,0.029326,0.198914


In [ ]:
#df_vec.to_csv('Word2Vec_Cbow_modelo_treinado',encoding='utf-8', index=False, header=True)

# 11. Naive Bayes + Word2Vec com CBOW

In [ ]:
label = preprocessing.LabelEncoder()

In [ ]:
label.fit(df_vec['sentimento'])
df_vec['sentimento'] = label.transform(df_vec['sentimento'])

In [ ]:
df_vec = df_vec.dropna()
df_vec

## Separando Treino e Teste

In [ ]:
target = df_vec['sentimento']

In [ ]:
feature = df_vec.iloc[:,1:50]

In [ ]:
feature

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.2, random_state=42)

## Avaliação do modelo

In [ ]:
clf = GaussianNB()

clf = clf.fit(X_train,y_train.values.ravel())

Y_pred = clf.predict(X_test)

print(classification_report(y_test, Y_pred))

In [ ]:
acc_score = accuracy_score(y_test, Y_pred)
format_output = "{:.2%}".format(acc_score)
print("Precisão final de :",format_output)

# 12. Word2Vec com embedding layer

## Definição de função

In [ ]:
from gensim.models import Word2Vec

# Função que treina o modelo Word2Vec no corpus do dataframe
def train_word2vec(df, column_name):
    # Obtém as frases tokenizadas
    sentences = df[column_name].tolist()

    # Treina o modelo Word2Vec
    model = Word2Vec(sentences, min_count=1)

    return model

In [ ]:
# Função que define os vetores para cada palavra do vocabulario
def get_word_vectors(model, sentence):
    vectors = []
    for word in sentence:
        if word in model.wv:
            vectors.append(model.wv[word]) # Append na lista de vetores
    if vectors:
        return np.sum(vectors, axis=0)/len(sentence) # Soma dos vetores para cada frase
    else:
        return np.zeros(model.vector_size)

# Criação do dataframe de vetores para cada frase
def create_word2vec_dataframe(df, column_name, model):
    sentences = df[column_name].tolist()
    vectors = [get_word_vectors(model, sentence) for sentence in sentences] # Itera para cada frase um vetor
    # Criação do dataframe
    df_vectors = pd.DataFrame(vectors, columns=[f"Vetor{i}" for i in range(model.vector_size)])
    df_word2vec = pd.concat([df, df_vectors], axis=1)
    return df_word2vec

## Teste de funções

In [ ]:
model = train_word2vec(df, 'texto_tratado')

In [ ]:
df_word2vec = create_word2vec_dataframe(df,'texto_tratado', model)
df_word2vec

In [ ]:
df_word2vec = df_word2vec.drop(columns=['autor', 'sentimento'])
df_word2vec

# 13. Naive Bayes + Word2Vec com embedding layer

In [ ]:
df_word2vec = df_word2vec.dropna()
df_word2vec

## Separando Treino e Teste

In [ ]:
target = df_word2vec['sentimentoNumerico']

In [ ]:
feature = df_word2vec.iloc[:,2:102]

In [ ]:
feature

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(feature, target, test_size=0.2, random_state=42)

## Avaliação do modelo

In [ ]:
clf = GaussianNB()

clf = clf.fit(X_train,y_train.values.ravel())

Y_pred = clf.predict(X_test)

print(classification_report(y_test, Y_pred))

In [ ]:
acc_score = accuracy_score(y_test, Y_pred)
format_output = "{:.2%}".format(acc_score)
print("Precisão final de :",format_output)

# XGBoost


## Criação do modelo


In [ ]:
encoder = LabelEncoder() # Instanciando o encoder para transformar as classes em números inteiros, a partir da biblioteca LabelEncoder

In [ ]:
import xgboost as xgb
# ajustando o codificador aos dados da coluna 'sentimento' para ser possível de se aplicar na predição
sentimento = encoder.fit_transform(df_vec["sentimento"])

# Dividir os dados em treino e teste, a partir da biblioteca de train_test_split
X_treino, X_teste, y_treino, y_teste = train_test_split(word2vec, sentimento, test_size=0.2, random_state=42)

# Criar e treinar o modelo XGBoost
modelo_xgb = xgb.XGBClassifier()

modelo_xgb.fit(X_treino, y_treino)

# Fazer a predição usando os dados de teste
predicao_numerica_xgb = modelo_xgb.predict(X_teste)

# Decodificar as classes preditas
predicao_xgb = encoder.inverse_transform(predicao_numerica_xgb)

print(classification_report(y_teste, predicao_xgb))

              precision    recall  f1-score   support

           0       0.49      0.50      0.50       360
           1       0.62      0.54      0.58       597
           2       0.60      0.66      0.63       651

    accuracy                           0.58      1608
   macro avg       0.57      0.57      0.57      1608
weighted avg       0.58      0.58      0.58      1608

